# Chapter 10: HTML5 APIs Overview

---

## Introduction

Modern web browsers are no longer just document viewers—they are sophisticated application platforms capable of storing data locally, accessing device sensors, manipulating browser history, and rendering complex graphics in real time. HTML5 introduced a suite of JavaScript APIs that enable web applications to rival native desktop and mobile applications in functionality.

These APIs allow developers to create offline-capable applications, location-aware services, smooth navigation experiences without page reloads, and dynamic visualizations. However, with these capabilities come important considerations: user privacy (especially with geolocation), security (with local storage), and performance (with canvas rendering).

In this chapter, you will learn how to harness the power of the Web Storage API for persistent data, the Geolocation API for location services, the History API for seamless navigation, and the Canvas API for programmatic graphics.

---

## 10.1 Web Storage API

The Web Storage API provides a way to store key-value pairs in the browser, persisting data beyond page refreshes and browser sessions. Unlike cookies, Web Storage offers significantly larger storage capacity (typically 5-10MB) and data is never transmitted to the server with HTTP requests.

### localStorage vs sessionStorage

Two storage mechanisms exist with different persistence models:

```javascript
// localStorage: Persists until explicitly deleted
localStorage.setItem('username', 'JohnDoe');
// Data survives browser restarts, tab closures, etc.

// sessionStorage: Persists only for the current tab/session
sessionStorage.setItem('tempData', '12345');
// Data is cleared when the tab/window is closed
```

**Comparison:**

```
┌─────────────────────────────────────────────────────────────────┐
│                  STORAGE MECHANISM COMPARISON                      │
├──────────────────┬─────────────────────┬────────────────────────┤
│ Feature          │ localStorage        │ sessionStorage         │
├──────────────────┼─────────────────────┼────────────────────────┤
│ Persistence      │ Until explicitly    │ Until tab/window       │
│                  │ cleared             │ closed                 │
├──────────────────┼─────────────────────┼────────────────────────┤
│ Scope            │ Origin-wide (all    │ Tab-specific           │
│                  │ tabs/windows)       │                        │
├──────────────────┼─────────────────────┼────────────────────────┤
│ Capacity         │ ~5-10 MB            │ ~5-10 MB               │
├──────────────────┼─────────────────────┼────────────────────────┤
│ Server transfer  │ Never               │ Never                  │
├──────────────────┼─────────────────────┼────────────────────────┤
│ Use case         │ User preferences,   │ Form drafts,           │
│                  │ auth tokens,        │ temporary state,       │
│                  │ shopping carts      │ multi-step forms       │
└──────────────────┴─────────────────────┴────────────────────────┘
```

### Storage Methods

**Setting data:**
```javascript
// Store string values
localStorage.setItem('theme', 'dark');
localStorage.setItem('fontSize', '16');

// Store numbers (converted to strings)
localStorage.setItem('visitCount', '0');

// Store objects (must serialize to JSON)
const user = { name: 'Alice', id: 123 };
localStorage.setItem('user', JSON.stringify(user));
```

**Retrieving data:**
```javascript
// Get values (returns null if key doesn't exist)
const theme = localStorage.getItem('theme'); // 'dark'
const missing = localStorage.getItem('nonexistent'); // null

// Parse JSON back to object
const userJson = localStorage.getItem('user');
const user = JSON.parse(userJson); // { name: 'Alice', id: 123 }

// Handle missing data with defaults
const settings = JSON.parse(localStorage.getItem('settings')) || {
    theme: 'light',
    notifications: true
};
```

**Removing data:**
```javascript
// Remove single item
localStorage.removeItem('tempData');

// Clear all storage for this origin
localStorage.clear();

// Remove specific item from session
sessionStorage.removeItem('formDraft');
```

**Checking storage:**
```javascript
// Get number of stored items
const itemCount = localStorage.length;

// Get key by index (useful for iteration)
for (let i = 0; i < localStorage.length; i++) {
    const key = localStorage.key(i);
    const value = localStorage.getItem(key);
    console.log(`${key}: ${value}`);
}
```

### Practical Example: Theme Preferences

```javascript
// Save user preference
function saveThemePreference(theme) {
    localStorage.setItem('theme', theme);
    document.documentElement.setAttribute('data-theme', theme);
}

// Load on page init
function loadThemePreference() {
    const savedTheme = localStorage.getItem('theme') || 'light';
    document.documentElement.setAttribute('data-theme', savedTheme);
    
    // Update UI to reflect saved choice
    const themeToggle = document.getElementById('theme-toggle');
    if (themeToggle) {
        themeToggle.checked = savedTheme === 'dark';
    }
}

// Usage
document.getElementById('theme-toggle').addEventListener('change', (e) => {
    const theme = e.target.checked ? 'dark' : 'light';
    saveThemePreference(theme);
});

// Initialize on page load
loadThemePreference();
```

### Storage Events

Changes to storage trigger events that can be listened to across tabs (for localStorage only):

```javascript
// Listen for storage changes (works across tabs)
window.addEventListener('storage', (event) => {
    console.log('Storage changed:', event.key);
    console.log('Old value:', event.oldValue);
    console.log('New value:', event.newValue);
    console.log('URL:', event.url); // Which page made the change
    
    // React to changes (e.g., sync theme across tabs)
    if (event.key === 'theme') {
        document.documentElement.setAttribute('data-theme', event.newValue);
    }
});
```

### Storage Limits and Error Handling

```javascript
function setWithExpiry(key, value, ttl) {
    const now = new Date();
    const item = {
        value: value,
        expiry: now.getTime() + ttl,
    };
    
    try {
        localStorage.setItem(key, JSON.stringify(item));
    } catch (e) {
        if (e.name === 'QuotaExceededError') {
            // Storage is full
            console.error('Storage quota exceeded');
            // Strategy: Clear old items or notify user
            handleStorageFull();
        }
    }
}

function getWithExpiry(key) {
    const itemStr = localStorage.getItem(key);
    if (!itemStr) return null;
    
    const item = JSON.parse(itemStr);
    const now = new Date();
    
    // Check if expired
    if (now.getTime() > item.expiry) {
        localStorage.removeItem(key);
        return null;
    }
    
    return item.value;
}

function handleStorageFull() {
    // Remove least recently used items or oldest data
    const keys = Object.keys(localStorage);
    // Sort by some priority or just remove first
    if (keys.length > 0) {
        localStorage.removeItem(keys[0]);
    }
}
```

**Important Security Note:**
```javascript
// NEVER store sensitive data
// ❌ Bad practice
localStorage.setItem('password', userPassword);
localStorage.setItem('jwtToken', secretToken); // XSS vulnerable

// ✅ Safer alternatives
// - HTTP-only cookies for auth tokens
// - Memory-only storage for sensitive session data
// - Encryption if absolutely necessary (with caution)
```

---

## 10.2 Geolocation API

The Geolocation API allows web applications to access the user's physical location with their explicit permission. This enables location-aware services like store finders, local weather, mapping, and delivery tracking.

### Getting Current Position

```javascript
if ('geolocation' in navigator) {
    navigator.geolocation.getCurrentPosition(
        (position) => {
            // Success callback
            const coords = position.coords;
            console.log('Latitude:', coords.latitude);
            console.log('Longitude:', coords.longitude);
            console.log('Accuracy:', coords.accuracy, 'meters');
            console.log('Altitude:', coords.altitude);
            console.log('Speed:', coords.speed);
            console.log('Heading:', coords.heading);
            
            displayMap(coords.latitude, coords.longitude);
        },
        (error) => {
            // Error callback
            handleGeolocationError(error);
        },
        {
            // Options
            enableHighAccuracy: true, // Use GPS if available (more battery)
            timeout: 10000,           // Wait max 10 seconds
            maximumAge: 60000         // Accept cached position up to 1 min old
        }
    );
} else {
    console.error('Geolocation not supported');
    fallbackToIPGeolocation();
}
```

**Position object structure:**
```
┌─────────────────────────────────────────────────────────────────┐
│                    POSITION OBJECT                                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  coords                                                          │
│  ├── latitude          (degrees)                                 │
│  ├── longitude         (degrees)                                 │
│  ├── accuracy          (meters, 95% confidence)                  │
│  ├── altitude          (meters above sea level, nullable)        │
│  ├── altitudeAccuracy  (meters, nullable)                        │
│  ├── heading           (degrees clockwise from north, nullable)  │
│  └── speed             (meters/second, nullable)                 │
│                                                                  │
│  timestamp             (DOMTimeStamp)                            │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### Watching Position (Continuous Updates)

For tracking movement in real-time:

```javascript
let watchId;

function startTracking() {
    watchId = navigator.geolocation.watchPosition(
        (position) => {
            updateUserLocation(position.coords);
        },
        (error) => {
            console.error('Tracking error:', error);
        },
        {
            enableHighAccuracy: true,
            maximumAge: 0,      // Always get fresh position
            timeout: 10000
        }
    );
}

function stopTracking() {
    if (watchId) {
        navigator.geolocation.clearWatch(watchId);
        watchId = null;
    }
}

// Clean up when page closes
window.addEventListener('beforeunload', stopTracking);
```

### Error Handling

```javascript
function handleGeolocationError(error) {
    switch(error.code) {
        case error.PERMISSION_DENIED:
            console.error('User denied geolocation permission');
            showMessage('Please enable location services to find nearby stores.');
            break;
        case error.POSITION_UNAVAILABLE:
            console.error('Location information unavailable');
            showMessage('Unable to determine your location. Please enter your zip code.');
            break;
        case error.TIMEOUT:
            console.error('Location request timed out');
            showMessage('Location request took too long. Please try again.');
            break;
        case error.UNKNOWN_ERROR:
            console.error('Unknown geolocation error');
            break;
    }
}
```

### Privacy and UX Best Practices

**Always request permission in context:**

```javascript
// ❌ Bad: Request on page load
window.onload = () => {
    navigator.geolocation.getCurrentPosition(...); // Annoying!
};

// ✅ Good: Request after user action
document.getElementById('find-nearby-btn').addEventListener('click', () => {
    // User clicked "Find Stores Near Me" - now it makes sense to ask
    navigator.geolocation.getCurrentPosition(
        (pos) => showNearbyStores(pos.coords),
        (err) => showZipCodeInput() // Fallback
    );
});
```

**Provide fallback options:**
```javascript
async function getLocation() {
    try {
        // Try GPS first
        const position = await new Promise((resolve, reject) => {
            navigator.geolocation.getCurrentPosition(resolve, reject);
        });
        return {
            lat: position.coords.latitude,
            lng: position.coords.longitude,
            source: 'gps'
        };
    } catch (error) {
        // Fallback to IP geolocation (less accurate)
        const response = await fetch('https://ipapi.co/json/');
        const data = await response.json();
        return {
            lat: data.latitude,
            lng: data.longitude,
            source: 'ip',
            city: data.city
        };
    }
}
```

---

## 10.3 History API

The History API allows manipulation of the browser's session history, enabling Single Page Applications (SPAs) to update the URL without page reloads, provide back/forward button functionality, and maintain state across navigation.

### Understanding History Stack

```
┌─────────────────────────────────────────────────────────────────┐
│                    BROWSER HISTORY STACK                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  [Home] → [Products] → [Product A] → [Product B] (current)      │
│                                      ↑                           │
│                                  User clicks back                │
│                                                                  │
│  [Home] → [Products] → [Product A] ← [Product B] (forward)      │
│                        ↑                                         │
│                      Now here                                    │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### pushState and replaceState

**`history.pushState()`**: Adds new entry to history stack
**`history.replaceState()`**: Modifies current entry without adding new one

```javascript
// State object, title (usually ignored), URL
history.pushState(
    { page: 'product', id: 123 },    // State data
    'Product Details',                // Title (unused in most browsers)
    '/products/123'                   // New URL (must be same origin)
);

// Replace current history entry (no back button to previous)
history.replaceState(
    { page: 'updated-product', id: 123 },
    'Updated Product',
    '/products/123-v2'
);
```

### Practical Example: SPA Navigation

```javascript
// Navigation handler
function navigateTo(url, state = {}) {
    // Update content via AJAX/fetch
    fetchContent(url).then(content => {
        // Update page content
        document.getElementById('app').innerHTML = content;
        
        // Update URL and history
        history.pushState(
            { url: url, ...state },
            document.title,
            url
        );
        
        // Update page title
        document.title = content.title || 'My App';
        
        // Scroll to top or restore position
        window.scrollTo(0, 0);
    });
}

// Handle browser back/forward buttons
window.addEventListener('popstate', (event) => {
    if (event.state) {
        // Restore previous state
        fetchContent(event.state.url).then(content => {
            document.getElementById('app').innerHTML = content;
            document.title = content.title;
        });
    } else {
        // Initial page load state
        loadInitialContent();
    }
});

// Intercept link clicks
document.addEventListener('click', (e) => {
    if (e.target.matches('a[href^="/"]')) {
        e.preventDefault();
        navigateTo(e.target.getAttribute('href'));
    }
});
```

### State Management

```javascript
// Save complex state
const appState = {
    filters: { category: 'electronics', price: '100-500' },
    sort: 'price-asc',
    pagination: { page: 2, perPage: 20 },
    scrollPosition: window.scrollY
};

history.pushState(appState, '', '/products?page=2');

// Restore state on popstate
window.addEventListener('popstate', (event) => {
    if (event.state) {
        applyFilters(event.state.filters);
        applySort(event.state.sort);
        restoreScrollPosition(event.state.scrollPosition);
    }
});
```

### Scroll Restoration

```javascript
// Save scroll position before navigation
let scrollPositions = {};

window.addEventListener('scroll', () => {
    scrollPositions[location.pathname] = window.scrollY;
});

// Restore on popstate
window.addEventListener('popstate', () => {
    const savedY = scrollPositions[location.pathname] || 0;
    window.scrollTo(0, savedY);
});
```

---

## 10.4 Canvas API

The Canvas API provides a 2D drawing context for rendering graphics, charts, animations, and image manipulation programmatically. Unlike SVG, which is resolution-independent and DOM-based, Canvas is pixel-based and immediate mode—once drawn, shapes become pixels and cannot be manipulated as objects.

### Setting Up Canvas

```html
<canvas id="myCanvas" width="800" height="600">
    <!-- Fallback content for browsers without canvas support -->
    <img src="fallback-chart.png" alt="Bar chart showing sales data">
</canvas>

<script>
const canvas = document.getElementById('myCanvas');
const ctx = canvas.getContext('2d'); // Get 2D rendering context

// Handle high DPI displays
const dpr = window.devicePixelRatio || 1;
const rect = canvas.getBoundingClientRect();

canvas.width = rect.width * dpr;
canvas.height = rect.height * dpr;
ctx.scale(dpr, dpr);
canvas.style.width = rect.width + 'px';
canvas.style.height = rect.height + 'px';
</script>
```

### Basic Drawing Operations

**Rectangles:**
```javascript
// Fill rectangle (x, y, width, height)
ctx.fillStyle = '#3498db';
ctx.fillRect(50, 50, 200, 100);

// Stroke (outline) rectangle
ctx.strokeStyle = '#2c3e50';
ctx.lineWidth = 3;
ctx.strokeRect(50, 50, 200, 100);

// Clear rectangle (erase)
ctx.clearRect(75, 75, 50, 50);
```

**Paths and Lines:**
```javascript
// Begin new path
ctx.beginPath();
ctx.moveTo(100, 100);  // Start point
ctx.lineTo(200, 100);  // Line to
ctx.lineTo(150, 200);  // Line to
ctx.closePath();       // Close shape

// Fill and stroke
ctx.fillStyle = '#e74c3c';
ctx.fill();
ctx.strokeStyle = '#c0392b';
ctx.stroke();

// Curves
ctx.beginPath();
ctx.moveTo(100, 300);
ctx.quadraticCurveTo(150, 200, 200, 300); // Control point, end point
ctx.stroke();

// Arcs and circles
ctx.beginPath();
ctx.arc(400, 300, 50, 0, Math.PI * 2); // x, y, radius, startAngle, endAngle
ctx.fillStyle = '#2ecc71';
ctx.fill();
```

**Text:**
```javascript
// Set font (CSS-style)
ctx.font = 'bold 24px Arial';
ctx.fillStyle = '#333';
ctx.textAlign = 'center';
ctx.textBaseline = 'middle';

// Fill text
ctx.fillText('Hello Canvas', 400, 100);

// Stroke text (outline)
ctx.strokeStyle = '#000';
ctx.lineWidth = 1;
ctx.strokeText('Hello Canvas', 400, 150);

// Measure text
const metrics = ctx.measureText('Hello');
console.log(metrics.width); // Width in pixels
```

### Drawing Images

```javascript
const img = new Image();
img.src = 'photo.jpg';
img.onload = () => {
    // Draw entire image (x, y)
    ctx.drawImage(img, 0, 0);
    
    // Draw scaled (x, y, width, height)
    ctx.drawImage(img, 0, 0, 400, 300);
    
    // Draw cropped section (sx, sy, sWidth, sHeight, dx, dy, dWidth, dHeight)
    ctx.drawImage(img, 100, 100, 200, 200, 0, 0, 400, 400);
};
```

### Animation Basics

```javascript
function animate() {
    // Clear canvas
    ctx.clearRect(0, 0, canvas.width, canvas.height);
    
    // Update positions
    x += velocity;
    
    // Draw frame
    ctx.fillRect(x, y, 50, 50);
    
    // Request next frame
    requestAnimationFrame(animate);
}

// Start animation
let x = 0, y = 100, velocity = 2;
animate();
```

### Accessibility for Canvas

Canvas is inherently inaccessible to screen readers. Always provide alternatives:

```html
<canvas id="chart" width="600" height="400" role="img" 
        aria-label="Bar chart showing 50% increase in Q4 sales">
</canvas>

<!-- Hidden but accessible table of data -->
<table class="visually-hidden">
    <caption>Q4 Sales Data</caption>
    <thead>
        <tr><th>Month</th><th>Sales</th></tr>
    </thead>
    <tbody>
        <tr><td>October</td><td>$100K</td></tr>
        <tr><td>November</td><td>$120K</td></tr>
        <tr><td>December</td><td>$150K</td></tr>
    </tbody>
</table>
```

Or use ARIA:

```javascript
// Update aria-label dynamically
canvas.setAttribute('aria-label', 'Chart showing 5 data points, highest value 95');
```

---

## Chapter Summary

In this chapter, you learned how to leverage HTML5 APIs to create dynamic, data-driven web applications:

1. **Web Storage API**: Use `localStorage` for persistent data (preferences, caching) and `sessionStorage` for temporary tab-specific data. Always serialize objects with `JSON.stringify()` and handle quota exceeded errors gracefully.

2. **Geolocation API**: Access user location with `getCurrentPosition()` for one-time lookups or `watchPosition()` for continuous tracking. Always request permission in context, handle errors gracefully, and provide manual entry fallbacks.

3. **History API**: Enable SPA-like navigation without page reloads using `pushState()` and `replaceState()`. Listen for `popstate` events to handle browser back/forward buttons properly. Store state objects to restore application state on navigation.

4. **Canvas API**: Draw 2D graphics, charts, and animations using paths, rectangles, text, and images. Remember that Canvas is pixel-based and requires fallback content for accessibility.

### Key Takeaways

- Never store sensitive authentication tokens in localStorage (vulnerable to XSS); use HTTP-only cookies instead
- Always check for API support before using (`if ('geolocation' in navigator)`)
- Request geolocation permission only in response to user action, not on page load
- When using History API, ensure server is configured to handle direct access to deep links
- Canvas content is not accessible to screen readers—always provide text alternatives or ARIA labels
- Storage limits vary by browser; implement error handling for `QuotaExceededError`

### Practice Exercises

1. Build a theme toggle that saves user preference to `localStorage` and syncs across browser tabs using the `storage` event.

2. Create a store locator that uses the Geolocation API to find the user's location, calculates distances to nearby stores, and displays them sorted by proximity. Include proper error handling and a manual zip code fallback.

3. Implement a simple Single Page Application router using the History API that updates content without page reloads, maintains browser history, and handles the back button correctly.

4. Create a Canvas-based drawing pad that allows users to draw with the mouse, clear the canvas, and save their drawing as a PNG image.

5. Build a data visualization component that uses Canvas to draw a bar chart from JSON data, with proper accessibility attributes describing the data for screen reader users.

---

## Coming Up Next

**Chapter 11: CSS Layouts — Positioning**

In the next chapter, we'll explore the fundamental CSS positioning schemes that control how elements are placed in the document flow. You'll learn:

- The `display` property and its values (`block`, `inline`, `inline-block`)
- CSS positioning schemes: `static`, `relative`, `absolute`, `fixed`, and `sticky`
- The stacking context and `z-index` management
- The `overflow` property and scrollable containers
- Understanding floats and clears (legacy but still relevant)

These positioning concepts form the foundation for all CSS layout techniques, from simple component positioning to complex overlay systems.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='9. multimedia_and_embedded_content.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../3. deep_dive_into_css/11. css_layouts_positioning.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
